# COMS W4705 Spring 24
## Homework 4 - Semantic Role Labelling with BERT

The goal of this assignment is to train and evaluate a PropBank-style semantic role labeling (SRL) system. Following (Collobert et al. 2011) and others, we will treat this problem as a sequence-labeling task. For each input token, the system will predict a B-I-O tag, as illustrated in the following example:

|The|judge|scheduled|to|preside|over|his|trial|was|removed|from|the|case|today|.|             
|---|-----|---------|--|-------|----|---|-----|---|-------|----|---|----|-----|-|             
|B-ARG1|I-ARG1|B-V|B-ARG2|I-ARG2|I-ARG2|I-ARG2|I-ARG2|O|O|O|O|O|O|O|
|||schedule.01|||||||||||||

Note that the same sentence may have multiple annotations for different predicates

|The|judge|scheduled|to|preside|over|his|trial|was|removed|from|the|case|today|.|             
|---|-----|---------|--|-------|----|---|-----|---|-------|----|---|----|-----|-|             
|B-ARG1|I-ARG1|I-ARG1|I-ARG1|I-ARG1|I-ARG1|I-ARG1|I-ARG1|O|B-V|B-ARG2|I-ARG2|I-ARG2|B-ARGM-TMP|O|
||||||||||remove.01||||||

and not all predicates need to be verbs

|The|judge|scheduled|to|preside|over|his|trial|was|removed|from|the|case|today|.|             
|---|-----|---------|--|-------|----|---|-----|---|-------|----|---|----|-----|-|    
|O|O|O|O|O|O|B-ARG1|B-V|O|O|O|O|O|O|O|
||||||||try.02||||||||

The SRL system will be implemented in [PyTorch](https://pytorch.org/). We will use BERT (in the implementation provided by the [Huggingface transformers](https://huggingface.co/docs/transformers/index) library) to compute contextualized token representations and a custom classification head to predict semantic roles. We will fine-tune the pretrained BERT model on the SRL task.


### Overview of the Approach

The model we will train is pretty straightforward. Essentially, we will just encode the sentence with BERT, then take the contextualized embedding for each token and feed it into a classifier to predict the corresponding tag.

Because we are only working on argument identification and labeling (not predicate identification), it is essentially that we tell the model where the predicate is. This can be accomplished in various ways. The approach we will choose here repurposes Bert's *segment embeddings*.

Recall that BERT is trained on two input sentences, seperated by [SEP], and on a next-sentence-prediction objective (in addition to the masked LM objective). To help BERT comprehend which sentence a given token belongs to, the original BERT uses a segment embedding, using A for the first sentene, and B for the second sentence 2.
Because we are labeling only a single sentence at a time, we can use the segment embeddings to indicate the predicate position instead: The predicate is labeled as segment B (1) and all other tokens will be labeled as segment A (0).

<img src="https://github.com/daniel-bauer/4705-f23-hw5/blob/main/bert_srl_model.png?raw=true" width=400px>

## Setup: GCP, Jupyter, PyTorch, GPU

To make sure that PyTorch is available and can use the GPU,run the following cell which should return True. If it doesn't, make sure the GPU drivers and CUDA are installed correctly.

GPU support is required for this assignment -- you will not be able to fine-tune BERT on a CPU.

In [ ]:
import torch
torch.cuda.is_available()

True

## Dataset: Ontonotes 5.0 English SRL annotations

We will work with the English part of the [Ontonotes 5.0](https://catalog.ldc.upenn.edu/LDC2013T19) data. This is an extension of PropBank, using the same type of annotation. Ontonotes contains annotations other than predicate/argument structures, but we will use the PropBank style SRL annotations only. *Important*: This data set is provided to you for use in COMS 4705 only! Columbia is a subscriber to LDC and is allowed to use the data for educational purposes. However, you may not use the dataset in projects unrelated to Columbia teaching or research.

If you haven't done so already, you can download the data here:


In [ ]:
! wget https://storage.googleapis.com/4705-bert-srl-data/ontonotes_srl.zip

--2024-12-11 22:53:48--  https://storage.googleapis.com/4705-bert-srl-data/ontonotes_srl.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.210.207, 173.194.215.207, 173.194.216.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.210.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12369688 (12M) [application/zip]
Saving to: ‘ontonotes_srl.zip’

ontonotes_srl.zip   100%[===================>]  11.80M  32.1MB/s    in 0.4s    

2024-12-11 22:53:49 (32.1 MB/s) - ‘ontonotes_srl.zip’ saved [12369688/12369688]



In [ ]:
! unzip ontonotes_srl.zip

Archive:  ontonotes_srl.zip
  inflating: propbank_dev.tsv        
  inflating: propbank_test.tsv       
  inflating: propbank_train.tsv      
  inflating: role_list.txt           


The data has been pre-processed in the following format. There are three files:

`propbank_dev.tsv`	`propbank_test.tsv`	`propbank_train.tsv`

Each of these files is in a tab-separated value format. A single predicate/argument structure annotation consists of four rows. For example

```
ontonotes/bc/cnn/00/cnn_0000.152.1
The     judge   scheduled       to      preside over    his     trial   was     removed from    the     case    today   /.
                schedule.01
B-ARG1  I-ARG1  B-V     B-ARG2  I-ARG2  I-ARG2  I-ARG2  I-ARG2  O       O       O       O       O       O       O
```

* The first row is a unique identifier (1st annotation of the 152nd sentence in the file ontonotes/bc/cnn/00/cnn_0000).
* The second row contains the tokens of the sentence (tab-separated).
* The third row contains the probank frame name for the predicate (empty field for all other tokens).
* The fourth row contains the B-I-O tag for each token.

The file `rolelist.txt` contains a list of propbank BIO labels in the dataset (i.e. possible output tokens). This list has been filtered to contain only roles that appeared more than 1000 times in the training data.
We will load this list and create mappings from numeric ids to BIO tags and back.

In [ ]:
role_to_id = {}
with open("role_list.txt",'r') as f:
    role_list = [x.strip() for x in f.readlines()]
    role_to_id = dict((role, index) for (index, role) in enumerate(role_list))
    role_to_id['[PAD]'] = -100

    id_to_role = dict((index, role) for (role, index) in role_to_id.items())


Note that we are also mapping the '[PAD]' token to the value -100. This allows the loss function to ignore these tokens during training.

## Part 1 - Data Preparation

Before you can build the SRL model, you first need to preprocess the data.


### 1.1 - Tokenization

One challenge is that the pre-trained BERT model uses subword ("WordPiece") tokenization, but the Ontonotes data does not. Fortunately Huggingface transformers provides a tokenizer.

In [ ]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenizer.tokenize("This is an unbelievably boring test sentence.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

['this',
 'is',
 'an',
 'un',
 '##bel',
 '##ie',
 '##va',
 '##bly',
 'boring',
 'test',
 'sentence',
 '.']

**TODO**:
We need to be able to maintain the correct labels (B-I-O tags) for each of the subwords.
Complete the following function that takes a list of tokens and a list of B-I-O labels of the same length as parameters, and returns a new token / label pair, as illustrated in the following example.


```
>>> tokenize_with_labels("the fancyful penguin devoured yummy fish .".split(), "B-ARG0 I-ARG0 I-ARG0 B-V B-ARG1 I-ARG1 O".split(), tokenizer)
(['the',
  'fancy',
  '##ful',
  'penguin',
  'dev',
  '##oured',
  'yu',
  '##mmy',
  'fish',
  '.'],
 ['B-ARG0',
  'I-ARG0',
  'I-ARG0',
  'I-ARG0',
  'B-V',
  'I-V',
  'B-ARG1',
  'I-ARG1',
  'I-ARG1',
  'O'])

```

To approach this problem, iterate through each word/label pair in the sentence. Call the tokenizer on the word. This may result in one or more tokens. Create the correct number of labels to match the number of tokens. Take care to not generate multiple B- tokens.


This approach is a bit slower than tokenizing the entire sentence, but is necessary to produce proper input tokenization for the pre-trained BERT model, and the matching target labels.

In [ ]:
def tokenize_with_labels(sentence, text_labels, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces.
    """

    tokenized = []
    labels = []

    for word, label in zip(sentence, text_labels):

      tokens = tokenizer.tokenize(word)
      tokenized.extend(tokens)

      if tokens:
        labels.append(label)

      # for subwords
      if len(tokens) > 1:
        base_label = 'I-' + label[2:] if label.startswith('B-') else label
        labels.extend([base_label] * (len(tokens) - 1))

    return tokenized, labels

In [ ]:
tokenize_with_labels("the fancyful penguin devoured yummy fish .".split(), "B-ARG0 I-ARG0 I-ARG0 B-V B-ARG1 I-ARG1 O".split(), tokenizer)

(['the',
  'fancy',
  '##ful',
  'penguin',
  'dev',
  '##oured',
  'yu',
  '##mmy',
  'fish',
  '.'],
 ['B-ARG0',
  'I-ARG0',
  'I-ARG0',
  'I-ARG0',
  'B-V',
  'I-V',
  'B-ARG1',
  'I-ARG1',
  'I-ARG1',
  'O'])

### 1.2 Loading the Dataset

Next, we are creating a PyTorch [Dataset](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) class. This class acts as a contained for the training, development, and testing data in memory. You should already be familiar with Datasets and Dataloaders from homework 3.

1.2.1 **TODO**: Write the \_\_init\_\_(self, filename) method that reads in the data from a data file (specified by the filename).

For each annotation you start with  the tokens in the sentence, and the BIO tags. Then you need to create the following

1. call the `tokenize_with_labels` function to tokenize the sentence.
2. Add the (token, label) pair to the self.items list.

1.2.2 **TODO**: Write the \_\_len\_\_(self) method that returns the total number of items.

1.2.3 **TODO**: Write the \_\_getitem\_\_(self, k) method that returns a single item in a format BERT will understand.
* We need to process the sentence by adding "\[CLS\]" as the first token and "\[SEP\]" as the last token. The need to pad the token sequence to 128 tokens using the "\[PAD\]" symbol. This needs to happen both for the inputs (sentence token sequence) and outputs (BIO tag sequence).
* We need to create an *attention mask*, which is a sequence of 128 tokens indicating the actual input symbols (as a 1) and \[PAD\] symbols (as a 0).
* We need to create a *predicate indicator* mask, which is a sequence of 128 tokens with at most one 1, in the position of the "B-V" tag. All other entries should be 0. The model will use this information to understand where the predicate is located.

* Finally, we need to convert the token and tag sequence into numeric indices. For the tokens, this can be done using the `tokenizer.convert_tokens_to_ids` method. For the tags, use the `role_to_id` dictionary.
Each sequence must be a pytorch tensor of shape (1,128). You can convert a list of integer values like this `torch.tensor(token_ids, dtype=torch.long)`.

To keep everything organized, we will return a dictionary in the following format

```
{'ids': token_tensor,
 'targets': tag_tensor,
 'mask': attention_mask_tensor,
 'pred': predicate_indicator_tensor}
```


(Hint: To debug these, read in the first annotation only / the first few annotations)


In [ ]:
from torch.utils.data import Dataset, DataLoader

class SrlData(Dataset):

    def __init__(self, filename):

        super(SrlData, self).__init__()

        self.max_len = 128 # the max number of tokens inputted to the transformer.

        self.tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', do_lower_case=True)

        self.items = []
        # complete this method
        curr = []
        i = 0

        with open(filename, 'r') as f:
          lines = f.readlines()

        while i < len(lines):
          # skip id
          i += 1
          if i >= len(lines):
            break

          tokens = lines[i].strip().split()
          #predicate frame
          i +=1
          if i >= len(lines):
            break

          # skip predicate frame
          i += 1
          if i >= len(lines):
            break

          labels = lines[i].strip().split()
          i += 1

          new_tokens, new_labels = tokenize_with_labels(tokens, labels, self.tokenizer)

          self.items.append((new_tokens, new_labels))



    def __len__(self):
        return len(self.items)

    def __getitem__(self, k):
      tokens, labels = self.items[k]

      # add [CLS] and [SEP]
      tokens = ['[CLS]'] + tokens + ['[SEP]']
      labels = ['O'] + labels + ['O']

      #init token tensor. convert tokens into ids.
      token_tensor = self.tokenizer.convert_tokens_to_ids(tokens)

      #init attention mask
      attn_mask = [1] * len(token_tensor)

      #init predicate indicator mask
      pred_tensor = [0] * len(token_tensor)
      for i, label in enumerate(labels):
        if label == 'B-V':
          pred_tensor[i] = 1

      #init label tensor. labels into ids.
      label_tensor = [role_to_id[label] if label in role_to_id else role_to_id['O'] for label in labels]

      pad_len = self.max_len - len(token_tensor)

      #padding
      token_tensor = token_tensor + [self.tokenizer.pad_token_id] * pad_len
      attn_mask = attn_mask + [0] * pad_len
      pred_tensor = pred_tensor + [0] * pad_len
      label_tensor = label_tensor + [-100] * pad_len

      # to tensor
      token_tensor = torch.tensor(token_tensor[:self.max_len], dtype=torch.long)
      attn_mask = torch.tensor(attn_mask[:self.max_len], dtype=torch.long)
      pred_tensor = torch.tensor(pred_tensor[:self.max_len], dtype=torch.long)
      label_tensor = torch.tensor(label_tensor[:self.max_len], dtype=torch.long)



      #complete this method
      return {'ids': token_tensor,
              'mask':  attn_mask,
              'targets': label_tensor,
              'pred': pred_tensor
              }


In [ ]:
# Reading the training data takes a while for the entire data because we preprocess all data offline
data = SrlData("propbank_train.tsv")

## 2. Model Definition

In [ ]:
from torch.nn import Module, Linear, CrossEntropyLoss
from transformers import BertModel

We will define the pyTorch model as a subclass of the [torch.nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html) class. The code for the model is provided for you. It may help to take a look at the documentation to remind you of how Module works. Take a look at how the huggingface BERT model simply becomes another sub-module.

In [ ]:
class SrlModel(Module):

    def __init__(self):

        super(SrlModel, self).__init__()

        self.encoder = BertModel.from_pretrained("bert-base-uncased")

        # The following two lines would freeze the BERT parameters and allow us to train the classifier by itself.
        # We are fine-tuning the model, so you can leave this commented out!
        # for param in self.encoder.parameters():
        #    param.requires_grad = False

        # The linear classifier head, see model figure in the introduction.
        self.classifier = Linear(768, len(role_to_id))


    def forward(self, input_ids, attn_mask, pred_indicator):

        # This defines the flow of data through the model

        # Note the use of the "token type ids" which represents the segment encoding explained in the introduction.
        # In our segment encoding, 1 indicates the predicate, and 0 indicates everything else.
        bert_output =  self.encoder(input_ids=input_ids, attention_mask=attn_mask, token_type_ids=pred_indicator)

        enc_tokens = bert_output[0] # the result of encoding the input with BERT
        logits = self.classifier(enc_tokens) #feed into the classification layer to produce scores for each tag.

        # Note that we are only interested in the argmax for each token, so we do not have to normalize
        # to a probability distribution using softmax. The CrossEntropyLoss loss function takes this into account.
        # It essentially computes the softmax first and then computes the negative log-likelihood for the target classes.
        return logits

In [ ]:
model = SrlModel().to('cuda') # create new model and store weights in GPU memory

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Now we are ready to try running the model with just a single input example to check if it is working correctly. Clearly it has not been trained, so the output is not what we expect. But we can see what the loss looks like for an initial sanity check.

**TODO**:
* Take a single data item from the dev set, as provided by your Dataset class defined above. Obtain the input token ids, attention mask, predicate indicator mask, and target labels.
* Run the model on the ids, attention mask, and predicate mask like this:

In [ ]:
# pick an item from the dataset. Then run

# first item picked
sample = data[0]
print(f"data: {sample}")

ids = sample['ids'].unsqueeze(0).cuda()
mask = sample['mask'].unsqueeze(0).cuda()
pred = sample['pred'].unsqueeze(0).cuda()
targets = sample['targets'].unsqueeze(0).cuda()

outputs = model(ids, mask, pred)
print(outputs)
print(outputs.shape)
print("\n")
print("id:", ids)
print("\n")
print("mask:", mask)
print("\n")
print("pred:", targets)
print("\n")
print("targets:", pred)

data: {'ids': tensor([  101,  2057, 26438,  2135, 13260,  2017,  2000,  3422,  1037,  2569,
         3179,  1997,  2408,  2859,  1012,   102,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

**TODO**:
Compute the loss on this one item only.
The initial loss should be close to -ln(1/num_labels)

Without training we would assume that all labels for each token (including the target label) are equally likely, so the negative log probability for the targets should be approximately $$-\ln(\frac{1}{\text{num_labels}}).$$ This is what the loss function should return on a single example. This is a good sanity check to run for any multi-class prediction problem.

In [ ]:
import math
-math.log(1 / len(role_to_id), math.e)

3.970291913552122

In [ ]:
loss_function = CrossEntropyLoss(ignore_index = -100, reduction='mean')

loss = loss_function(outputs.transpose(2,1), targets)
print(loss.item())


# complete this. Note that you still have to provide a (batch_size, input_pos)
# tensor for each parameter, where batch_size =1

# outputs = model(ids, mask, pred)
# loss = loss_function(...)
# loss.item()   #this should be approximately the score from the previous cell


4.269010066986084


**TODO**: At this point you should also obtain the actual predictions by taking the argmax over each position.
The result should look something like this (values will differ).

```
tensor([[ 1,  4,  4,  4,  4,  4,  5, 29, 29, 29,  4, 28,  6, 32, 32, 32, 32, 32,
         32, 32, 30, 30, 32, 30, 32,  4, 32, 32, 30,  4, 49,  4, 49, 32, 30,  4,
         32,  4, 32, 32,  4,  2,  4,  4, 32,  4, 32, 32, 32, 32, 30, 32, 32, 30,
         32,  4,  4, 49,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  6,  6, 32, 32,
         30, 32, 32, 32, 32, 32, 30, 30, 30, 32, 30, 49, 49, 32, 32, 30,  4,  4,
          4,  4, 29,  4,  4,  4,  4,  4,  4, 32,  4,  4,  4, 32,  4, 30,  4, 32,
         30,  4, 32,  4,  4,  4,  4,  4, 32,  4,  4,  4,  4,  4,  4,  4,  4,  4,
          4,  4]], device='cuda:0')
```

Then use the id_to_role dictionary to decode to actual tokens.

```
['[CLS]', 'O', 'O', 'O', 'O', 'O', 'B-ARG0', 'I-ARG0', 'I-ARG0', 'I-ARG0', 'O', 'B-V', 'B-ARG1', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG1', 'I-ARG1', 'I-ARG2', 'I-ARG1', 'I-ARG2', 'O', 'I-ARG2', 'I-ARG2', 'I-ARG1', 'O', 'I-ARGM-TMP', 'O', 'I-ARGM-TMP', 'I-ARG2', 'I-ARG1', 'O', 'I-ARG2', 'O', 'I-ARG2', 'I-ARG2', 'O', '[SEP]', 'O', 'O', 'I-ARG2', 'O', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG1', 'I-ARG2', 'I-ARG2', 'I-ARG1', 'I-ARG2', 'O', 'O', 'I-ARGM-TMP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ARG1', 'B-ARG1', 'I-ARG2', 'I-ARG2', 'I-ARG1', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG2', 'I-ARG1', 'I-ARGM-TMP', 'I-ARGM-TMP', 'I-ARG2', 'I-ARG2', 'I-ARG1', 'O', 'O', 'O', 'O', 'I-ARG0', 'O', 'O', 'O', 'O', 'O', 'O', 'I-ARG2', 'O', 'O', 'O', 'I-ARG2', 'O', 'I-ARG1', 'O', 'I-ARG2', 'I-ARG1', 'O', 'I-ARG2', 'O', 'O', 'O', 'O', 'O', 'I-ARG2', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
```

For now, just make sure you understand how to do this for a single example. Later, you will write a more formal function to do this once we have trained the model.

In [ ]:
predictions = torch.argmax(outputs, dim=2)
print("Predictions:", predictions)


pred_labels = []
for pred_id in predictions[0].cpu().numpy():
    if pred_id in id_to_role:
        pred_labels.append(id_to_role[pred_id])
    else:
        pred_labels.append('O')

print("Decoded labels:", pred_labels)

Predictions: tensor([[ 5, 31, 40,  5, 28,  1,  1, 45, 30,  1, 11, 21, 21, 11, 45,  3, 16, 16,
         16, 24, 24, 44, 44, 44, 44, 24, 44, 44, 44, 30, 30, 44, 44, 30, 30, 44,
          4, 44, 44, 44, 17, 17, 44,  8, 24, 24, 44, 16, 44, 44, 44, 16, 16, 44,
         44, 24, 44, 14, 30, 30, 30,  4,  4, 44,  7, 21, 11, 17, 17, 44, 44, 24,
         24, 44, 24, 44, 44, 44, 44, 40, 44, 44, 44, 44, 40, 24, 14, 44, 30, 44,
         16,  4, 44,  7, 11, 11, 44, 17,  8, 24, 24, 24, 24, 24, 24, 44, 44, 44,
         24, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 11, 40, 44,
         44, 44]], device='cuda:0')
Decoded labels: ['B-ARG0', 'I-ARG1-DSP', 'I-ARGM-EXT', 'B-ARG0', 'B-V', '[CLS]', '[CLS]', 'I-ARGM-NEG', 'I-ARG1', '[CLS]', 'B-ARGM-ADJ', 'B-ARGM-NEG', 'B-ARGM-NEG', 'B-ARGM-ADJ', 'I-ARGM-NEG', '[prd]', 'B-ARGM-EXT', 'B-ARGM-EXT', 'B-ARGM-EXT', 'B-ARGM-PRR', 'B-ARGM-PRR', 'I-ARGM-MOD', 'I-ARGM-MOD', 'I-ARGM-MOD', 'I-ARGM-MOD', 'B-ARGM-PRR', 'I-ARGM-MOD', 'I-ARGM-MOD', 'I-ARGM-MOD', 

## 3. Training loop

pytorch provides a DataLoader class that can be wrapped around a Dataset to easily use the dataset for training. The DataLoader allows us to easily adjust the batch size and shuffle the data.

In [ ]:
from torch.utils.data import DataLoader
loader = DataLoader(data, batch_size = 32, shuffle = True)

The following cell contains the main training loop. The code should work as written and report the loss after each batch,
cumulative average loss after each 100 batches, and print out the final average loss after the epoch.

**TODO**: Modify the training loop belowso that it also computes the accuracy for each batch and reports the
average accuracy after the epoch.
The accuracy is the number of correctly predicted token labels out of the number of total predictions.
Make sure you exclude [PAD] tokens, i.e. tokens for which the target label is -100. It's okay to include [CLS] and [SEP] in the accuracy calculation.

In [ ]:
loss_function = CrossEntropyLoss(ignore_index = -100, reduction='mean')

LEARNING_RATE = 1e-05
optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE)

device = 'cuda'

def train():
    """
    Train the model for one epoch.
    """
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()

    count_correct = 0 # total correct predictions
    count_predictions = 0 # total predictions

    for idx, batch in enumerate(loader):

        # Get the encoded data for this batch and push it to the GPU
        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)
        pred_mask = batch['pred'].to(device, dtype = torch.long)

        # Run the forward pass of the model
        logits = model(input_ids=ids, attn_mask=mask, pred_indicator=pred_mask)
        loss = loss_function(logits.transpose(2,1), targets)
        tr_loss += loss.item()
        print("Batch loss: ", loss.item()) # can comment out if too verbose.

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)

        #accuracy calculation
        mask = targets != -100
        pred = torch.argmax(logits, dim=2)
        correct = ((pred == targets) & mask).sum().item()
        total = mask.sum().item()

        count_correct += correct
        count_predictions += total


        if idx % 100==0:
            #torch.cuda.empty_cache() # can help if you run into memory issues
            curr_avg_loss = tr_loss/nb_tr_steps
            print(f"Current average loss: {curr_avg_loss}")

        # Compute accuracy for this batch
        matching = torch.sum(torch.argmax(logits,dim=2) == targets)
        predictions = torch.sum(torch.where(targets==-100,0,1))

        # Run the backward pass to update parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    epoch_acc = count_correct / count_predictions
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {epoch_acc}")

Now let's train the model for one epoch. This will take a while (up to a few hours).

In [ ]:
train()

Streaming output truncated to the last 5000 lines.
Batch loss:  0.25523364543914795
Batch loss:  0.5757360458374023
Batch loss:  0.24277359247207642
Batch loss:  0.6346496939659119
Batch loss:  0.2344309538602829
Batch loss:  0.14691494405269623
Batch loss:  0.2653692662715912
Batch loss:  0.2206350564956665
Batch loss:  0.22528070211410522
Batch loss:  0.13483379781246185
Batch loss:  0.1634998470544815
Batch loss:  0.33663541078567505
Batch loss:  0.3714222311973572
Batch loss:  0.6717811226844788
Batch loss:  0.25702810287475586
Batch loss:  0.20928527414798737
Batch loss:  0.2195415049791336
Batch loss:  0.14833861589431763
Batch loss:  0.25381776690483093
Batch loss:  0.427974134683609
Batch loss:  0.23078159987926483
Batch loss:  0.5748152732849121
Batch loss:  0.27916061878204346
Batch loss:  0.3163420855998993
Batch loss:  0.39263713359832764
Batch loss:  0.2016090303659439
Batch loss:  0.23023320734500885
Batch loss:  0.23513758182525635
Batch loss:  0.2967531383037567
Batch l

In my experiments, I found that two epochs are needed for good performance.

In [ ]:
train()

Streaming output truncated to the last 5000 lines.
Batch loss:  0.16584597527980804
Batch loss:  0.27934813499450684
Batch loss:  0.09817025810480118
Batch loss:  0.24744169414043427
Batch loss:  0.1330053210258484
Batch loss:  0.2068357765674591
Batch loss:  0.18350078165531158
Batch loss:  0.1082337349653244
Batch loss:  0.1497410088777542
Batch loss:  0.1866641789674759
Batch loss:  0.1454104483127594
Batch loss:  0.17736485600471497
Batch loss:  0.09435809403657913
Batch loss:  0.13129793107509613
Batch loss:  0.1709567904472351
Batch loss:  0.07053215801715851
Batch loss:  0.13685278594493866
Batch loss:  0.24282872676849365
Batch loss:  0.32076919078826904
Batch loss:  0.22556981444358826
Batch loss:  0.2260591834783554
Batch loss:  0.2269153892993927
Batch loss:  0.1663934737443924
Batch loss:  0.21081708371639252
Batch loss:  0.19700568914413452
Batch loss:  0.1930190622806549
Batch loss:  0.2609268128871918
Batch loss:  0.1186586543917656
Batch loss:  0.13805101811885834
Batch

I ended up with a training loss of about 0.19 and a training accuracy of 0.94. Specific values may differ.

At this point, it's a good idea to save the model (or rather the parameter dictionary) so you can continue evaluating the model without having to retrain.

In [ ]:
torch.save(model.state_dict(), "srl_model_fulltrain_2epoch_finetune_1e-05.pt")

## 4. Decoding

In [ ]:
# Optional step: If you stopped working after part 3, first load the trained model

model = SrlModel().to('cuda')
model.load_state_dict(torch.load("srl_model_fulltrain_2epoch_finetune_1e-05.pt"))
model = model.to('cuda')

<ipython-input-37-56056f4e4de6>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("srl_model_fulltrain_2epoch_finetune_1e-05.pt"))


**TODO (this is the fun part)**: Now that we have a trained model, let's try labeling an unseen example sentence. Complete the functions decode_output and label_sentence below. decode_output takes the logits returned by the model, extracts the argmax to obtain the label predictions for each token, and then translate the result into a list of string labels.

label_sentence takes a list of input tokens and a predicate index, prepares the model input, call the model and then call decode_output to produce a final result.

Note that you have already implemented all components necessary (preparing the input data from the token list and predicate index, decoding the model output). But now you are putting it together in one convenient function.

In [ ]:
tokens = "A U. N. team spent an hour inside the hospital , where it found evident signs of shelling and gunfire .".split()

In [ ]:
def decode_output(logits): # it will be useful to have this in a separate function later on
    """
    Given the model output, return a list of string labels for each token.
    """
    pred = torch.argmax(logits, dim=2)[0].cpu().numpy()

    #prediction id to labels
    labels = []
    for pred_id in pred:
        if pred_id in id_to_role:
            labels.append(id_to_role[pred_id])
        else:
            labels.append('O')

    return labels


In [ ]:
def label_sentence(tokens, pred_idx):
    # complete this function to prepare token_ids, attention mask, predicate mask, then call the model.
    # Decode the output to produce a list of labels.
    print(f"input tokens: {tokens}")

    #prepare input
    tokenized = []
    token_map = []
    output_labels = []


    for i, word in enumerate(tokens):
      subtokens = tokenizer.tokenize(word)
      tokenized.extend(subtokens)
      token_map.extend([i] * len(subtokens)) #map position of subtokens

    bert_tokens = ['[CLS]'] + tokenized + ['[SEP]']
    token_map = [-1] + token_map + [-1] # cls and sep token pos set to -1 in token map

    print(f"Tokenized: {tokenized}")
    print(f"Token Map: {token_map}")

    # to ids
    token_ids = tokenizer.convert_tokens_to_ids(bert_tokens)

    # create attention mask
    attn_mask = [1] * len(token_ids)

    # create predicate indicator
    pred_tensor = [0] * len(token_ids)
    # 1 at the first subtoken of pred token
    for i, original_position in enumerate(token_map):
      if original_position == pred_idx:
        pred_tensor[i] = 1
        break

    # padding
    max_len = 128
    pad_len = max_len - len(token_ids)

    token_tensor = token_ids + [tokenizer.pad_token_id] * pad_len
    attn_mask = attn_mask + [0] * pad_len
    pred_tensor = pred_tensor + [0] * pad_len

    # to tensors
    token_tensor = torch.tensor(token_tensor[:max_len], dtype=torch.long).unsqueeze(0).cuda()
    attn_mask = torch.tensor(attn_mask[:max_len], dtype=torch.long).unsqueeze(0).cuda()
    pred_tensor = torch.tensor(pred_tensor[:max_len], dtype=torch.long).unsqueeze(0).cuda()


    # predictions
    model.eval()
    with torch.no_grad():
        logits = model(input_ids=token_tensor, attn_mask=attn_mask, pred_indicator=pred_tensor)

    # decode predictions
    labels = decode_output(logits)

    print(f"labels {labels}")



    # map back to original tokens
    output_labels = []
    for i in range(len(tokens)):
        for j, orig_idx in enumerate(token_map):
            if orig_idx == i:
                output_labels.append(labels[j])
                break

    print(pred_tensor)
    print(bert_tokens)

    return output_labels





In [ ]:
# Now you should be able to run

label_test = label_sentence(tokens, 13) # Predicate is "found"
zip(tokens, label_test)

input tokens: ['A', 'U.', 'N.', 'team', 'spent', 'an', 'hour', 'inside', 'the', 'hospital', ',', 'where', 'it', 'found', 'evident', 'signs', 'of', 'shelling', 'and', 'gunfire', '.']
Tokenized: ['a', 'u', '.', 'n', '.', 'team', 'spent', 'an', 'hour', 'inside', 'the', 'hospital', ',', 'where', 'it', 'found', 'evident', 'signs', 'of', 'shell', '##ing', 'and', 'gunfire', '.']
Token Map: [-1, 0, 1, 1, 2, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 17, 18, 19, 20, -1]
labels ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ARGM-LOC', 'I-ARGM-LOC', 'O', 'B-ARGM-LOC', 'B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-ARGM-LOC', 'B-ARG0', 'O', 'O', 'B-ARG0', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ARGM-LOC', 'I-ARGM-LOC', 'O', 'O', 'B-ARG0', 'O', 'B

In [ ]:
print(list(zip(tokens, label_test)))

[('A', 'O'), ('U.', 'O'), ('N.', 'O'), ('team', 'O'), ('spent', 'O'), ('an', 'O'), ('hour', 'O'), ('inside', 'O'), ('the', 'B-ARGM-LOC'), ('hospital', 'I-ARGM-LOC'), (',', 'O'), ('where', 'B-ARGM-LOC'), ('it', 'B-ARG0'), ('found', 'B-V'), ('evident', 'B-ARG1'), ('signs', 'I-ARG1'), ('of', 'I-ARG1'), ('shelling', 'I-ARG1'), ('and', 'I-ARG1'), ('gunfire', 'I-ARG1'), ('.', 'O')]


The expected output is somethign like this:
```   
 ('A', 'O'),
 ('U.', 'O'),
 ('N.', 'O'),
 ('team', 'O'),
 ('spent', 'O'),
 ('an', 'O'),
 ('hour', 'O'),
 ('inside', 'O'),
 ('the', 'B-ARGM-LOC'),
 ('hospital', 'I-ARGM-LOC'),
 (',', 'O'),
 ('where', 'B-ARGM-LOC'),
 ('it', 'B-ARG0'),
 ('found', 'B-V'),
 ('evident', 'B-ARG1'),
 ('signs', 'I-ARG1'),
 ('of', 'I-ARG1'),
 ('shelling', 'I-ARG1'),
 ('and', 'I-ARG1'),
 ('gunfire', 'I-ARG1'),
 ('.', 'O'),
```


### 5. Evaluation 1: Token-Based Accuracy
We want to evaluate the model on the dev or test set.

In [ ]:
dev_data = SrlData("propbank_dev.tsv") # Takes a while because we preprocess all data offline

In [ ]:
from torch.utils.data import DataLoader
loader = DataLoader(dev_data, batch_size = 1, shuffle = False)

In [ ]:
# Optional: Load the model again if you stopped working prior to this step.
#model = SrlModel()
#model.load_state_dict(torch.load("srl_model_fulltrain_2epoch_finetune_1e-05.pt"))
#model = mode.to('cuda')

**TODO**: Complete the evaluate_token_accuracy function below. The function should iterate through the items in the data loader (see training loop in part 3). Run the model on each sentence/predicate pair and extract the predictions.

For each sentence, count the correct predictions and the total predictions. Finally, compute the accuracy as #correct_predictions / #total_predictions

Careful: You need to filter out the padded positions ([PAD] target tokens), as well as [CLS] and [SEP]. It's okay to include [B-V] in the count though.

In [ ]:
def evaluate_token_accuracy(model, loader):

    model.eval() # put model in evaluation mode

    # for the accuracy
    total_correct = 0  # number of correct token label predictions.
    total_predictions = 0 # number of total predictions = number of tokens in the data.

    # iterate over the data here.
    with torch.no_grad():
        for batch in loader:
            ids = batch['ids'].to('cuda', dtype=torch.long)
            mask = batch['mask'].to('cuda', dtype=torch.long)
            targets = batch['targets'].to('cuda', dtype=torch.long)
            pred_mask = batch['pred'].to('cuda', dtype=torch.long)

            # foward pass
            logits = model(input_ids=ids, attn_mask=mask, pred_indicator=pred_mask)
            pred_labels = decode_output(logits)

            predictions = torch.tensor(
                [role_to_id[label] if label in role_to_id else role_to_id['O']
                 for label in pred_labels],
                device='cuda'
            )

            #filter out pad, cls, sep
            cls_token_id = tokenizer.cls_token_id
            sep_token_id = tokenizer.sep_token_id
            mask = (targets != -100) & (ids != cls_token_id) & (ids != sep_token_id)

            correct = ((predictions == targets) & mask).sum().item()
            total = mask.sum().item()

            total_correct += correct
            total_predictions += total

    acc = total_correct / total_predictions
    print(f"Accuracy: {acc}")


In [ ]:
evaluate_token_accuracy(model,loader)

Accuracy: 0.9331217188097759


### 6. Span-Based evaluation

While the accuracy score in part 5 is encouraging, an accuracy-based evaluation is problematic for two reasons. First, most of the target labels are actually O. Second, it only tells us that per-token prediction works, but does not directly evaluate the SRL performance.

Instead, SRL systems are typically evaluated on micro-averaged precision, recall, and F1-score for predicting labeled spans.

More specifically, for each sentence/predicate input, we run the model, decode the output, and extract a set of labeled spans (from the output and the target labels). These spans are (i,j,label) tuples.  

We then compute the true_positives, false_positives, and false_negatives based on these spans.

In the end, we can compute

* Precision:  true_positive / (true_positives + false_positives)  , that is the number of correct spans out of all predicted spans.

* Recall: true_positives / (true_positives + false_negatives) , that is the number of correct spans out of all target spans.

* F1-score:   (2 * precision * recall) / (precision + recall)


For example, consider

| |[CLS]|The|judge|scheduled|to|preside|over|his|trial|was|removed|from|the|case|today|.|             
|--||---|-----|---------|--|-------|----|---|-----|---|-------|----|---|----|-----|-|             
||0|1|2|3|4|5|6|7|8|9|1O|11|12|13|14|15|
|target|[CLS]|B-ARG1|I-ARG1|B-V|B-ARG2|I-ARG2|I-ARG2|I-ARG2|I-ARG2|O|O|O|O|O|O|O|
|prediction|[CLS]|B-ARG1|I-ARG1|B-V|I-ARG2|I-ARG2|O|O|O|O|O|O|O|O|B-ARGM-TMP|O|

The target spans are (1,2,"ARG1"), and (4,8,"ARG2").

The predicted spans would be (1,2,"ARG1"), (14,14,"ARGM-TMP"). Note that in the prediction, there is no proper ARG2 span because we are missing the B-ARG2 token, so this span should not be created.

So for this sentence we woudl get: true_positives: 1 false_positives: 1 false_negatives: 1

*TODO*: Complete the function evaluate_spans that performs the span-based evaluation on the given model and data loader. You can use the provided extract_spans function, which returns the spans as a dictionary. For example
{(1,2): "ARG1", (4,8):"ARG2"}

In [ ]:
def extract_spans(labels):
    spans = {} # map (start,end) ids to label
    current_span_start = 0
    current_span_type = ""
    inside = False
    for i, label in enumerate(labels):
        if label.startswith("B"):
            if inside:
                if current_span_type != "V":
                    spans[(current_span_start,i)] = current_span_type
            current_span_start = i
            current_span_type = label[2:]
            inside = True
        elif inside and label.startswith("O"):
            if current_span_type != "V":
                spans[(current_span_start,i)] = current_span_type
            inside = False
        elif inside and label.startswith("I") and label[2:] != current_span_type:
            if current_span_type != "V":
                spans[(current_span_start,i)] = current_span_type
            inside = False
    return spans


In [ ]:
def evaluate_spans(model, loader):
    model.eval()
    total_tp = 0
    total_fp = 0
    total_fn = 0

    with torch.no_grad():
        for batch in loader:
            ids = batch['ids'].to('cuda', dtype=torch.long)
            mask = batch['mask'].to('cuda', dtype=torch.long)
            targets = batch['targets'].to('cuda', dtype=torch.long)
            pred_mask = batch['pred'].to('cuda', dtype=torch.long)

            logits = model(input_ids=ids, attn_mask=mask, pred_indicator=pred_mask)

            for i in range(ids.size(0)):
                pred_labels = decode_output(logits[i:i+1])

                # filter out cls, sep, and pad tokens
                valid_mask = ((targets[i] != -100) &
                            (ids[i] != tokenizer.cls_token_id) &
                            (ids[i] != tokenizer.sep_token_id))

                valid_indices = valid_mask.nonzero().squeeze(-1)

                target_labels = [id_to_role[targets[i][idx].item()]
                               for idx in valid_indices]


                pred_labels = [pred_labels[idx.item()]
                             for idx in valid_indices]

                pred_spans = extract_spans(pred_labels)
                gold_spans = extract_spans(target_labels)

                for span, label in pred_spans.items():
                    if span in gold_spans and gold_spans[span] == label:
                        total_tp += 1
                    else:
                        total_fp += 1

                for span, label in gold_spans.items():
                    if span not in pred_spans or pred_spans[span] != label:
                        total_fn += 1

    total_p = total_tp / (total_tp + total_fp)
    total_r = total_tp / (total_tp + total_fn)
    total_f = (2 * total_p *total_r) / (total_p + total_r)

    print(f"Overall P: {total_p}  Overall R: {total_r}  Overall F1: {total_f}")

    return total_p, total_r, total_f

In [ ]:
evaluate_spans(model, loader)

Overall P: 0.8289990962494351  Overall R: 0.8256134468908565  Overall F1: 0.8273028077315491


(0.8289990962494351, 0.8256134468908565, 0.8273028077315491)

In my evaluation, I got an F score of 0.82  (which slightly below the state-of-the art in 2018)

### OPTIONAL:

Repeat the span-based evaluation, but print out precision/recall/f1-score for each role separately.